# Computing Fundemental Statistics

In this section, you will learn to compute essential statistical measures such as: 
- Sum 
- Mean 
- Median
- Quantiles

For each method, we will compare the actual values to the computed differentially private values to demonstrate utility. The [documentation](https://docs.opendp.org/en/nightly/api/python/opendp.polars.html#module-opendp.polars) also provides more information about the methods. We will use the [sample data](https://github.com/opendp/dp-test-datasets/blob/master/data/sample_FR_LFS.csv.zip) from the Labour Force Survey in France. 

## Set Up

In [4]:
%pip install numpy matplotlib seaborn 
%pip install "opendp[polars]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import polars as pl 
import opendp.prelude as dp
import seaborn as sns 

dp.enable_features("contrib")
sns.set_theme(style='darkgrid')

In [6]:
df = pl.scan_csv("sample_FR_LFS.csv")

The compositor here is nearly identical to the one explained in the introduction. One additional parameter that is included is the `max_num_partitions` which is required when the metric isn't sensitive to ordering. In the following examples, there is only one partition in select, so `max_num_partitions` is set to 1. 

In [7]:
# Filter HWUSUAL of null values. 
df = df.filter(pl.col("HWUSUAL") != 99)

estimated_max_partition_len = 60_000_000

context = dp.Context.compositor(
    data=df,
    privacy_unit=dp.unit_of(contributions=36),
    privacy_loss=dp.loss_of(epsilon=1.0),
    split_evenly_over=5,
    margins={
        (): dp.Margin(public_info= "lengths", max_partition_length=estimated_max_partition_len, max_num_partitions=1),
    },
)

In these examples we seem some differences between how regular Polars and Polars with DP are used:
### 1. Specifying the Data

In a regular query, use `df` directly. In a differentially private query, use `context.query()`. 

### 2. Applying the Method

In a regular query, you'll often be able to apply the function directly, such as `.sum()`. In a differentially private query, you'll specify the package and pass the required parameters. For example, to compute the differentially private sum, pass in bounds as a tuple for example, `dp.sum((1,10))`. The bounds are the bounds of the input data, essentially the known minimum and maximum. 

### 3. Collecting the Results 

In a regular query, use `collect()`. In a differentially private query, use `release().collect().`. 

## Sum

To demonstrate the `sum` method, calculate the total number of hours worked for all years in the dataset. 

This mirrors [aggregrate hours](https://www.investopedia.com/terms/a/aggregate_hours.asp), a statistic gathered by the U.S. Department of Labor that represents the total hours worked by all people during a year. 

In [8]:
total_hours_actual = df.select(pl.col("HWUSUAL").sum()).collect().item()
print('Actual Total Hours: ', total_hours_actual)

Actual Total Hours:  2962104.0


The query for the sum is essentially the same. We add an extra argument, `fill_null` because the `sum` method requires a non-nullable input. 

Do not use private data to calculate imputed values or bounds: This could leak private information, and rendering the differential privacy guarantees meaningless. Instead, choose bounds and imputed values based on prior domain knowledge.

In [9]:
total_hours_dp = context.query().select(pl.col("HWUSUAL").fill_null(0.).dp.sum((1,80))).release().collect().item()
print('Differentially Private Total Hours: ', total_hours_dp)

Differentially Private Total Hours:  2953689.33103334


## Mean

To demonstrate the `mean` method, calculate the mean number of hours worked for all years in the entire dataset. The `mean` method also requires all null values to be filled. The bounds parameter is the same as the bounds parameter used in `sum`. 

In [10]:
mean_hours_actual = df.select(pl.col("HWUSUAL").mean()).collect().item()
print('Actual mean hours: ', mean_hours_actual)

Actual mean hours:  37.63409056258576


In [11]:
mean_hours_dp = context.query().select(pl.col("HWUSUAL").fill_null(40.).dp.mean((1,80))).release().collect().item()
print('Differentially Private Mean for Hours: ', mean_hours_dp )

Differentially Private Mean for Hours:  37.72491125272871


## Median


To demonstrate the `median` method, calculate the median number of hours worked for all years in the entire dataset. The `median` method requires a parameter `candidates,` which is the potential value for the median. 

In [12]:
median_hours_actual = df.select(pl.col("HWUSUAL").median()).collect().item()
print('Actual Hours Median: ', median_hours_actual)

Actual Hours Median:  37.0


In [13]:
median_candidates = list(range(20,60))
median_hours_dp = context.query().select(pl.col("HWUSUAL").fill_null(40.).dp.median(median_candidates)).release().collect().item()
print('Differentially Private Median for Hours: ', median_hours_dp )

Differentially Private Median for Hours:  37


## Quantiles

To demonstrate the `quantile` method, calculate the number of hours worked at the 25th percentile. 

The `median` method requires two parameters:
- Quantile: This is between 0 and 1. We provide 0.25. 
- Candidates: A range of possible values for your quantiles. We provide values from 20 through 60.  

In [14]:
quantile = 0.25
quantile_30_actual = df.select(pl.col("HWUSUAL").quantile(quantile)).collect().item()
print('Actual 25th Quantile for Hours: ', quantile_30_actual)

Actual 25th Quantile for Hours:  35.0


In [15]:
quantile_candidates = list(range(20, 60))
quantile_30_dp = context.query().select(pl.col("HWUSUAL").fill_null(40.).dp.quantile(quantile, quantile_candidates)).release().collect().item()
print('Differentially Private 25th Quantile for Hours: ', quantile_30_dp)

Differentially Private 25th Quantile for Hours:  35


### Computing Multiple Quantiles

In [16]:
multiple_quantiles_actual = df.select(
    [pl.col("HWUSUAL").fill_null(40.).quantile(q).alias(f"Quantile_{q}") for q in [0.2, 0.4, 0.6, 0.8]]
).collect()
multiple_quantiles_actual

Quantile_0.2,Quantile_0.4,Quantile_0.6,Quantile_0.8
f64,f64,f64,f64
34.0,35.0,39.0,44.0


In [17]:
multiple_quantiles_dp = context.query().select(
    [pl.col("HWUSUAL").fill_null(40.).dp.quantile(q, quantile_candidates).alias(f"Quantile_{q}") for q in [0.2, 0.4, 0.6, 0.8]]
).release().collect()
multiple_quantiles_dp

Quantile_0.2,Quantile_0.4,Quantile_0.6,Quantile_0.8
i64,i64,i64,i64
34,36,38,44
